In [3]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import shutil
import pandas as pd
from PIL import Image
import torch
import json
import csv

In [100]:

def split_img(img_path, save_dir):
    img = cv2.imread(img_path)
    h, w, _ = img.shape
    
    step_x = int(w / 3)
    step_y = int(h / 3)
    overlap = 70

    num_of_slice_w = (w + overlap) // (step_x - overlap)
    num_of_slice_h = (h + overlap) // (step_y - overlap)

    slices_2d = []
    for i in range(num_of_slice_w):
        row_slices = []
        for j in range(num_of_slice_h):
            start_x = i * (step_x - overlap)
            start_y = j * (step_y - overlap)
            end_x = min(start_x + step_x, w)
            end_y = min(start_y + step_y, h)

            img_slice = img[start_y:end_y, start_x:end_x]
            img_name = img_path[-7:]

            slice_filename = f'{img_name[:-4]}_w{i}_h{j}.bmp'
            print(slice_filename)
            print(save_dir)
            cv2.imwrite(os.path.join(save_dir, slice_filename), img_slice)

            row_slices.append({'slice': slice_filename, 'bboxes': [], 'start_x': start_x, 'start_y': start_y}) 
        
        slices_2d.append(row_slices)

    return slices_2d

def is_bbox_in_slice(bbox, start_x, start_y, end_x, end_y):
    _, tl_x, tl_y, bl_x, bl_y, tr_x, tr_y, br_x, br_y = bbox
    tl_x, tl_y, bl_x, bl_y, tr_x, tr_y, br_x, br_y = float(tl_x), float(tl_y), float(bl_x), float(bl_y), float(tr_x), float(tr_y), float(br_x), float(br_y)
    if (start_x <= tl_x <= end_x and start_y <= tl_y <= end_y and
        start_x <= bl_x <= end_x and start_y <= bl_y <= end_y and
        start_x <= tr_x <= end_x and start_y <= tr_y <= end_y and
        start_x <= br_x <= end_x and start_y <= br_y <= end_y):
        return True
    return False

def distribute_bboxes(bboxes, slices_2d, img_w, img_h, step_x, step_y, overlap):
    for bbox in bboxes:
        for i, row in enumerate(slices_2d):
            for j, img_slice_data in enumerate(row):

                start_x = img_slice_data['start_x']
                start_y = img_slice_data['start_y']
                end_x = min(start_x + step_x, img_w)
                end_y = min(start_y + step_y, img_h)

                if is_bbox_in_slice(bbox, start_x, start_y, end_x, end_y):
                    new_bbox = [
                        bbox[0],
                        float(bbox[1]) - start_x, 
                        float(bbox[2]) - start_y, 
                        float(bbox[3]) - start_x, 
                        float(bbox[4]) - start_y, 
                        float(bbox[5]) - start_x,  
                        float(bbox[6]) - start_y,  
                        float(bbox[7]) - start_x,  
                        float(bbox[8]) - start_y  
                    ]
                    img_slice_data['bboxes'].append(new_bbox)


def save_slices_and_bboxes_csv(slices_2d, save_dir, name):
    csv_file = os.path.join(save_dir, f"{name[:-4]}.csv")
    
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['_Slice', "id",'Bbox_Top_Left_X', 'Bbox_Top_Left_Y', 
                         'Bbox_Bottom_Left_X', 'Bbox_Bottom_Left_Y', 
                         'Bbox_Bottom_Right_X', 'Bbox_Bottom_Right_Y',
                         'Bbox_Top_Right_X', 'Bbox_Top_Right_Y'
                         ])

        for i, row in enumerate(slices_2d):
            for j, img_slice_data in enumerate(row):
                slice_filename = img_slice_data['slice']
                for bbox in img_slice_data['bboxes']:
                    writer.writerow([slice_filename] + bbox)


In [101]:

def prep_preprocess(img_path, save_dir):
    for folder in os.listdir(img_path):
        fold_path = os.path.join(img_path, folder)
        temp = ""

        for f_path in os.listdir(fold_path):
            file_path = os.path.join(fold_path, f_path)
            if file_path.endswith(".bmp"):
                img = cv2.imread(file_path)
                if img is None:
                    print(f"Error loading image: {file_path}")
                    continue

                h, w, _ = img.shape
                
                slices_2d = split_img(file_path, save_dir)
                temp = f_path  
            #########################################################
            #########################################################
            elif file_path.endswith(".csv") and temp[:-4] == f_path[:-4]:
                bboxes = pd.read_csv(file_path, header=None)
                bboxes = bboxes.values.tolist()
                bboxes = bboxes[1:]  

                distribute_bboxes(bboxes, slices_2d, w, h, step_x=w//3, step_y=h//3, overlap=50)

                save_slices_and_bboxes_csv(slices_2d, save_dir, temp)

path = r"D:\Work\codev2\data\final_data\final_19.9v1"
save_dir = r"D:\Work\codev2\code\test_data"

prep_preprocess(path, save_dir)


053_w0_h0.bmp
D:\Work\codev2\code\test_data
053_w0_h1.bmp
D:\Work\codev2\code\test_data
053_w0_h2.bmp
D:\Work\codev2\code\test_data
053_w1_h0.bmp
D:\Work\codev2\code\test_data
053_w1_h1.bmp
D:\Work\codev2\code\test_data
053_w1_h2.bmp
D:\Work\codev2\code\test_data
053_w2_h0.bmp
D:\Work\codev2\code\test_data
053_w2_h1.bmp
D:\Work\codev2\code\test_data
053_w2_h2.bmp
D:\Work\codev2\code\test_data
019_w0_h0.bmp
D:\Work\codev2\code\test_data
019_w0_h1.bmp
D:\Work\codev2\code\test_data
019_w0_h2.bmp
D:\Work\codev2\code\test_data
019_w1_h0.bmp
D:\Work\codev2\code\test_data
019_w1_h1.bmp
D:\Work\codev2\code\test_data
019_w1_h2.bmp
D:\Work\codev2\code\test_data
019_w2_h0.bmp
D:\Work\codev2\code\test_data
019_w2_h1.bmp
D:\Work\codev2\code\test_data
019_w2_h2.bmp
D:\Work\codev2\code\test_data
067_w0_h0.bmp
D:\Work\codev2\code\test_data
067_w0_h1.bmp
D:\Work\codev2\code\test_data
067_w0_h2.bmp
D:\Work\codev2\code\test_data
067_w1_h0.bmp
D:\Work\codev2\code\test_data
067_w1_h1.bmp
D:\Work\codev2\cod

In [102]:


def csv_to_labelme_json(csv_file, output_dir):
    df = pd.read_csv(csv_file)
    
    grouped = df.groupby(['_Slice'])
    list_black_label = ["053", "019", "067", "068"]
    

    for image_name, group in grouped:
        image_name = str(image_name)
        # print(image_name[2:-3])
        if str(image_name[2:5]) in list_black_label:
            label = "black_box"
        else:
            label = "numb_box"
        # print(label)
        # continue  
        labelme_data = {
            "version": "4.5.7",
            "flags": {},
            "shapes": [],
            "imagePath": image_name[2:-3],
            "imageData": None,
            "imageHeight": None,
            "imageWidth": None
        }
        
        for _, row in group.iterrows():
            bbox = [
                row['Bbox_Top_Left_X'],
                row['Bbox_Top_Left_Y'],
                row['Bbox_Bottom_Right_X'] - row['Bbox_Top_Left_X'],
                row['Bbox_Bottom_Right_Y'] - row['Bbox_Top_Left_Y']
            ]
            shape = {
                "label": label,
                "points": [[row['Bbox_Top_Left_X'], row['Bbox_Top_Left_Y']],
                        [row['Bbox_Bottom_Right_X'], row['Bbox_Top_Left_Y']],
                        [row['Bbox_Bottom_Right_X'], row['Bbox_Bottom_Right_Y']],
                        [row['Bbox_Top_Left_X'], row['Bbox_Bottom_Right_Y']]],
                "group_id": None,
                "shape_type": "polygon",
                "flags": {}
            }
            labelme_data['shapes'].append(shape)
        
        labelme_data["imageHeight"] = 1216   
        labelme_data["imageWidth"] = 1824   
        json_file = os.path.join(output_dir, image_name[2:-7]+ ".json")
        with open(json_file, 'w') as f:
            json.dump(labelme_data, f, indent=4)

save_dir = r"D:\Work\codev2\code\test_data"
for f_name in os.listdir(save_dir):
    if f_name.endswith(".csv"):
        csv_path = os.path.join(save_dir, f_name)  
        print(f_name)
        csv_to_labelme_json(csv_path, save_dir)


000.csv
001.csv
003.csv
006.csv
008.csv
017.csv
018.csv
019.csv
021.csv
023.csv
026.csv
029.csv
039.csv
040.csv
041.csv
042.csv
043.csv
044.csv
045.csv
046.csv
047.csv
049.csv
050.csv
051.csv
052.csv
053.csv
056.csv
065.csv
067.csv
068.csv
069.csv
090.csv
095.csv
098.csv
099.csv
107.csv
109.csv
110.csv
112.csv
113.csv
114.csv
115.csv
116.csv


In [54]:
im_path = r"D:\Work\codev2\code\test_data\001_w2_h2.bmp"
img = cv2.imread(im_path)
w, h, ch = img.shape
print(w, h)

1216 1824


In [89]:
list_black_label = ["053", "019", "067", "068"]

if '019' in list_black_label:
    print("Chuỗi '019' có trong danh sách.")
else:
    print("Chuỗi '019' không có trong danh sách.")


Chuỗi '019' có trong danh sách.
